# Model Notebook

This notebook is only used to help speed up data cleaning/transformations and model creation process.

In [1]:
import math
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

field_height = 53.3
field_length = 120
endzone_length = 10
seconds_in_a_half = 15 * 60
seconds_in_overtime = 10 * 60

In [2]:
start = time.time()
games = pd.read_csv('data/games.csv')
print("Done in:", time.time() - start)

Done in: 0.0047130584716796875


In [3]:
start = time.time()
players = pd.read_csv('data/players.csv')
print("Done in:", time.time() - start)

Done in: 0.011034250259399414


In [4]:
start = time.time()
plays = pd.read_csv('data/plays.csv')
print("Done in:", time.time() - start)

Done in: 0.0747981071472168


In [5]:
start = time.time()
plays = plays.merge(games, on='gameId')
print("Done in:", time.time() - start)

plays['is_home_offense'] = plays['possessionTeam'] == plays['homeTeamAbbr']
print("Done in:", time.time() - start)

Done in: 0.026874065399169922
Done in: 0.029808759689331055


In [6]:
start = time.time()
scout = pd.read_csv('data/PFFScoutingData.csv')
print("Done in:", time.time() - start)

Done in: 0.04279780387878418


In [7]:
start = time.time()
tracking_2018 = pd.read_csv('data/tracking2018.csv')
tracking_2019 = pd.read_csv('data/tracking2019.csv')
tracking_2020 = pd.read_csv('data/tracking2020.csv')
tracking = tracking_2018.append([tracking_2019, tracking_2020])
print("Done in:", time.time() - start)

Done in: 69.28465628623962


In [8]:
start = time.time()
left_mask = tracking['playDirection'] == 'left'
right_mask = tracking['playDirection'] == 'right'
tracking.loc[left_mask, 'stdX'] = field_length - tracking.loc[left_mask, 'x']
tracking.loc[left_mask, 'stdY'] = field_height - tracking.loc[left_mask, 'y']
tracking.loc[left_mask, 'stdDir'] = (360 - tracking.loc[left_mask, 'dir']) % 360
tracking.loc[right_mask, 'stdX'] = tracking.loc[right_mask, 'x']
tracking.loc[right_mask, 'stdY'] = tracking.loc[right_mask, 'y']
tracking.loc[right_mask, 'stdDir'] = tracking.loc[right_mask, 'dir']
tracking['stdR'] = (tracking['stdX'].pow(2) + tracking['stdY'].pow(2)).pow(1/2)
tracking['stdTheta'] = np.arctan(tracking['stdY']/tracking['stdX'])
print("Done in:", time.time() - start)

Done in: 19.19552779197693


In [9]:
num_games = plays['gameId'].unique().size

start = time.time()
play_results = plays.groupby(['specialTeamsPlayType', 'specialTeamsResult']).size().unstack('specialTeamsPlayType')
play_results = play_results.divide(num_games)
play_results_probs = play_results.div(play_results.sum(axis=0), axis=1)
excitement_scores = play_results_probs.subtract(1).multiply(-1).unstack().dropna().rename('excitement_score_v1')
plays = plays.join(excitement_scores, on=['specialTeamsPlayType', 'specialTeamsResult'])
print("Done in:", time.time() - start)

start = time.time()
plays.loc[plays['is_home_offense'], 'score_diff'] = plays['preSnapHomeScore'] - plays['preSnapVisitorScore']
plays.loc[~plays['is_home_offense'], 'score_diff'] = plays['preSnapVisitorScore'] - plays['preSnapHomeScore']
plays['score_diff_ex'] = pd.cut(plays['score_diff'].abs(), 
    bins=[-1, 0, 1, 3, 5, 8, 11, 16, 21, 26, 31], 
    labels=[.9, 1, .8, .7, .6, .5, .4, .3, .2, .1]
).astype(float)
plays['excitement_score_v2'] = 0.5 * plays['excitement_score_v1'] + 0.5 * plays['score_diff_ex']
print("Done in:", time.time() - start)

start = time.time()
plays['gameClockMins'] = plays['gameClock'].str.split(':').str[0]
plays['gameClockSecs'] = plays['gameClock'].str.split(':').str[1]
plays['seconds_left_in_quarter'] = plays['gameClockMins'].astype(int) * 60 + plays['gameClockSecs'].astype(int)
plays.loc[plays['quarter'].isin([2, 4, 5]), 'seconds_left_in_half'] = plays['seconds_left_in_quarter']
plays.loc[plays['quarter'].isin([1, 3]), 'seconds_left_in_half'] = plays['seconds_left_in_quarter'] + seconds_in_a_half
print("Done in:", time.time() - start)

start = time.time()
plays.loc[plays['quarter'] != 5, 'time_left_ex'] = (1 - plays['seconds_left_in_half'] / (seconds_in_a_half * 2))
plays.loc[plays['quarter'] == 5, 'time_left_ex'] = (1 - plays['seconds_left_in_half'] / seconds_in_overtime)
plays['quarter_ex'] = plays['quarter'] / 5
plays['excitement_score_v3'] = 0.25 * (plays['excitement_score_v1'] + plays['score_diff_ex'] + plays['quarter_ex'] + plays['time_left_ex'])
print("Done in:", time.time() - start)

Done in: 0.08814620971679688
Done in: 0.020440101623535156
Done in: 0.08430886268615723
Done in: 0.00700688362121582


In [10]:
kickoffs = plays[plays['specialTeamsPlayType']=='Kickoff'].reset_index(drop=True)

start = time.time()
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
transformed = encoder.fit_transform(kickoffs[['specialTeamsResult']])
output_cols = [x.split('_')[1] for x in encoder.get_feature_names()]
ohe_df = pd.DataFrame(transformed, columns=output_cols)
kickoffs = pd.concat([kickoffs, ohe_df], axis=1)
print("Done in:", time.time() - start)

start = time.time()
kickoffs['initial_excitement'] = 1/3 * (kickoffs['score_diff_ex'] + kickoffs['quarter_ex'] + kickoffs['time_left_ex'])
kickoff_tracking = kickoffs.merge(tracking, on=['gameId', 'playId'])
print("Done in:", time.time() - start)

start = time.time()
kickoff_tracking['is_off'] = (kickoff_tracking['is_home_offense'] & (kickoff_tracking['team'] == 'home')) | (~kickoff_tracking['is_home_offense'] & (kickoff_tracking['team'] != 'home'))
print("Done in:", time.time() - start)

Done in: 0.012713909149169922
Done in: 101.9402801990509
Done in: 1.3968241214752197


In [11]:
# kickoff_tracking = kickoff_tracking[(kickoff_tracking['gameId']==2018090600) & (kickoff_tracking['playId'].isin([37, 1387]))]
# kickoff_tracking.head()

In [12]:
print(kickoff_tracking.columns)
kickoff_tracking.head()

Index(['gameId', 'playId', 'playDescription', 'quarter', 'down', 'yardsToGo',
       'possessionTeam', 'specialTeamsPlayType', 'specialTeamsResult',
       'kickerId', 'returnerId', 'kickBlockerId', 'yardlineSide',
       'yardlineNumber', 'gameClock', 'penaltyCodes', 'penaltyJerseyNumbers',
       'penaltyYards', 'preSnapHomeScore', 'preSnapVisitorScore', 'passResult',
       'kickLength', 'kickReturnYardage', 'playResult',
       'absoluteYardlineNumber', 'season', 'week', 'gameDate',
       'gameTimeEastern', 'homeTeamAbbr', 'visitorTeamAbbr', 'is_home_offense',
       'excitement_score_v1', 'score_diff', 'score_diff_ex',
       'excitement_score_v2', 'gameClockMins', 'gameClockSecs',
       'seconds_left_in_quarter', 'seconds_left_in_half', 'time_left_ex',
       'quarter_ex', 'excitement_score_v3', 'Downed', 'Fair Catch',
       'Kickoff Team Recovery', 'Muffed', 'Out of Bounds', 'Return',
       'Touchback', 'initial_excitement', 'time', 'x', 'y', 's', 'a', 'dis',
       'o', 'di

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,specialTeamsPlayType,specialTeamsResult,kickerId,...,position,team,frameId,playDirection,stdX,stdY,stdDir,stdR,stdTheta,is_off
0,2018090600,37,J.Elliott kicks 65 yards from PHI 35 to end zo...,1,0,0,PHI,Kickoff,Touchback,44966.0,...,FS,home,1,right,43.76,8.10,53.28,44.503344,0.183029,True
1,2018090600,37,J.Elliott kicks 65 yards from PHI 35 to end zo...,1,0,0,PHI,Kickoff,Touchback,44966.0,...,FS,home,2,right,43.77,8.10,73.05,44.513177,0.182988,True
2,2018090600,37,J.Elliott kicks 65 yards from PHI 35 to end zo...,1,0,0,PHI,Kickoff,Touchback,44966.0,...,FS,home,3,right,43.78,8.10,70.39,44.523010,0.182947,True
3,2018090600,37,J.Elliott kicks 65 yards from PHI 35 to end zo...,1,0,0,PHI,Kickoff,Touchback,44966.0,...,FS,home,4,right,43.80,8.11,71.53,44.544496,0.183086,True
4,2018090600,37,J.Elliott kicks 65 yards from PHI 35 to end zo...,1,0,0,PHI,Kickoff,Touchback,44966.0,...,FS,home,5,right,43.84,8.12,75.17,44.585648,0.183143,True


The only features we're interested in right now is location of players and the ball. So we'll selected those columns and standardize (scale) them.

In [13]:
features = ['stdR', 'stdTheta']

sc = MinMaxScaler()

start = time.time()
kickoff_tracking[features] = sc.fit_transform(kickoff_tracking[features])
print("Done in:", time.time() - start)

Done in: 29.97202777862549


# Need to rename playerIds to 0-11 in order to optimize space (and time) since this step is failing!!
vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv


In [14]:
# start = time.time()
# df = kickoff_tracking.set_index(['gameId', 'playId', 'frameId', 'is_off', 'nflId']).sort_index(level=['is_off', 'frameId', 'nflId'])[features].unstack(['is_off', 'nflId']).reset_index(['gameId', 'playId', 'frameId'])
# print("Done in:", time.time() - start)

# df.head()

In [15]:
start = time.time()
df = kickoff_tracking.set_index(['gameId', 'playId', 'frameId', 'is_off', 'nflId']).sort_index(level=['gameId', 'playId','frameId', 'is_off', 'nflId']).droplevel('nflId')
df = df.set_index(df.groupby(['gameId', 'playId', 'frameId']).cumcount().rename('playerNum'), append=True)
df = df[features].unstack(['is_off', 'playerNum']).reset_index(['gameId', 'playId', 'frameId'])
print("Done in:", time.time() - start)

df.tail(50)

Done in: 160.2979609966278


gameId playId frameId      stdR                                \
is_off                                   False                                 
playerNum                                    0         1         2         3   
618048     2021010315   3886      54  0.599991  0.709529  0.618113  0.639264   
618049     2021010315   3886      55  0.603289  0.710428  0.620317  0.641340   
618050     2021010315   3886      56  0.606636  0.711311  0.622393  0.643319   
618051     2021010315   3886      57  0.609894  0.712180  0.624347  0.645178   
618052     2021010315   3886      58  0.613153  0.713079  0.626272  0.646919   
618053     2021010315   3886      59  0.616302  0.713986  0.628071  0.648562   
618054     2021010315   4182       1  0.347062  0.382382  0.471346  0.379773   
618055     2021010315   4182       2  0.347388  0.382463  0.471364  0.379857   
618056     2021010315   4182       3  0.347799  0.382487  0.471364  0.379870   
618057     2021010315   4182       4  0.348270  0.382498  0.471364  0.379949   
618058     2021010315   4182       5  0.348863  0.382510  0.471306  0.379957   
618059     2021010315   4182       6  0.349676  0.382522  0.471328  0.379966   
618060     2021010315   4182       7  0.350902  0.382545  0.471234  0.379975   
618061     2021010315   4182       8  0.352554  0.382581  0.471162  0.379975   
618062     2021010315   4182       9  0.354911  0.382651  0.471073  0.379900   
618063     2021010315   4182      10  0.357757  0.382747  0.470908  0.379895   
618064     2021010315   4182      11  0.361156  0.382937  0.470710  0.379829   
618065     2021010315   4182      12  0.365054  0.383316  0.470495  0.379688   
618066     2021010315   4182      13  0.369541  0.383818  0.470109  0.379626   
618067     2021010315   4182      14  0.374445  0.384443  0.469592  0.379573   
618068     2021010315   4182      15  0.379720  0.385100  0.469037  0.379670   
618069     2021010315   4182      16  0.385195  0.385912  0.468272  0.379702   
618070     2021010315   4182      17  0.390872  0.386859  0.467468  0.379820   
618071     2021010315   4182      18  0.396557  0.387930  0.466625  0.379943   
618072     2021010315   4182      19  0.402227  0.388937  0.465759  0.380149   
618073     2021010315   4182      20  0.407800  0.390076  0.464911  0.380367   
618074     2021010315   4182      21  0.413312  0.391199  0.464247  0.380593   
618075     2021010315   4182      22  0.418675  0.392394  0.463600  0.380905   
618076     2021010315   4182      23  0.423829  0.393665  0.463153  0.381090   
618077     2021010315   4182      24  0.428785  0.394933  0.462822  0.381294   
618078     2021010315   4182      25  0.433722  0.396149  0.462577  0.381509   
618079     2021010315   4182      26  0.438454  0.397439  0.462532  0.381611   
618080     2021010315   4182      27  0.443063  0.398669  0.462545  0.381721   
618081     2021010315   4182      28  0.447107  0.400009  0.462646  0.381784   
618082     2021010315   4182      29  0.450813  0.401295  0.462863  0.381857   
618083     2021010315   4182      30  0.454292  0.402533  0.463094  0.381878   
618084     2021010315   4182      31  0.457431  0.403728  0.463328  0.381830   
618085     2021010315   4182      32  0.460031  0.404948  0.463400  0.381863   
618086     2021010315   4182      33  0.462546  0.406169  0.463589  0.381827   
618087     2021010315   4182      34  0.464266  0.407326  0.463606  0.381729   
618088     2021010315   4182      35  0.466085  0.408622  0.463761  0.381622   
618089     2021010315   4182      36  0.466955  0.409848  0.463732  0.381512   
618090     2021010315   4182      37  0.468135  0.411093  0.463546  0.381330   
618091     2021010315   4182      38  0.468210  0.412355  0.463387  0.381144   
618092     2021010315   4182      39  0.468850  0.413537  0.463086  0.380953   
618093     2021010315   4182      40  0.468527  0.414665  0.462842  0.380885   
618094     2021010315   4182      41  0.468825  0.415839  0.462528  0.380811   
618095     2021

Then we'll convert the DataFrame to a 2D array and remove all the `nan` values, since those are for players that are not in the play. The first column is the **gameId**, second is **playId**, and third is **frameId**. This will probably be useful once we implement a recurrent neural net (which can remember previous input), but that's probably pretty far down the road.

In [16]:
start = time.time()
values = np.array([x[~np.isnan(x)] for x in df.values])
print("Done in:", time.time() - start)
values.shape

Done in: 2.057347059249878


(618098, 49)

In [17]:
start = time.time()
output = kickoff_tracking.groupby(['gameId', 'playId', 'frameId'])[output_cols].first().values
print("Done in:", time.time() - start)
output.shape

Done in: 2.4998631477355957


(618098, 7)